First we check the GPU version available in the environment and install specific dependencies that are compatible with the detected GPU to prevent version conflicts.

In [3]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

Load model


In [5]:
from unsloth import FastLanguageModel

max_seq_length = 6000 # Choose any! Llama 3 is up to 8k
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/Colab/lora_model", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2024.7 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Generate response


In [6]:
# the system prompt
alpaca_prompt = """Genereaza o ofertă detaliată pentru client pe baza cererii:

### Cererea:
{}

### Oferta:
{}"""

FastLanguageModel.for_inference(model)

input_description = """\nAutomatizare documente in ERP unde sa primeasca automat urmatorul nr de serie si sa se tina cont de firma producatoare sau client; Au licenta achizitioata de la ONEDRIVE. 2. Realizare baza de date (import excel) in 2 categorii : cumparatori si furnizori; 3. Produse : adaugate din excel cu anul de productie, cod de identificare, denumire, paametrii de calitate lot ( doresc un filtru pe acest modul) 4. Cantitate : transporta doar TONE si este de 3 feluri : cantitate efectiva, cantitate tolerate (5, 10%) si cantitate la optiunea cuiva (comparator sau furnizor) 5. Livrare conditii: - incoterms („Incoterms” reprezintă prescurtarea pentru International Commercial Terms. Publicată pentru prima dată în 1936, reprezintă de fapt un set de 11 reguli, care definesc responsabilitatea în cadrul tranzacțiilor internaționale.) - Locatie ( aici se doreste onlista principala automatizata si posibilitatea de a adauga manual ) deoarece anumiti furnizori ofera mereu transport pana intr-un punct(acelasi gen vama) de acolo revine in responsailitatea clientului 6. Perioada de livrare : aleasa fie de comparator fie de furnizor. 7. Pret: sunt de 2 feluri. Fix stabilit din perioada de negociere si semnare ctr in 3 valute RON, USD si EUR. Pret matif ( este un pret la o data certa). Adica se semneaza ctr cu pret matif din data de… 8. Termen de plata : automatizare daca clientul e din RO pt moneda ( programul sa puna automat in LEI) si selectare manuala a perioadei sau sugestii de genul 30 zile. Pt ctr negociate diverse variante in functie de produs, origine si transport. 9. Comision la partea de ctr ( pt fiecare ctr inregistrat sa se adauge comisionul perceput si din partea cui, furnizor, comparator sau ambii si agentul care se ocupa ). Sa existe posibilitatea de a face update pe fiecare ctr cu status exact de genul : documente trimis apoi document semnat. Automatizare comisioane in functie de setarile initiale cu cantitatea pe care se factureaza si moneda. 10. Raportare : pe contract, pe furnizor si pe comparator. 11. Facturi: evidenta facturi parteneri, in special pt ca facturile de la furnizori trec in registrul lor de evidente si sa poata observa cand un client a achitat. Pe cantitati, pret, valori daca cumva se poate automatiza ceva. Evidenta facturi proprii, lucreaza cu SMARTBILL dar vor schimba de la 1 Ian cu E-FACTRA."""

inputs = tokenizer(
[
    alpaca_prompt.format(
        input_description, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 6000, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Genereaza o ofertă detaliată pentru client pe baza cererii:\n\n### Cererea:\n\nAutomatizare documente in ERP unde sa primeasca automat urmatorul nr de serie si sa se tina cont de firma producatoare sau client; Au licenta achizitioata de la ONEDRIVE. 2. Realizare baza de date (import excel) in 2 categorii : cumparatori si furnizori; 3. Produse : adaugate din excel cu anul de productie, cod de identificare, denumire, paametrii de calitate lot ( doresc un filtru pe acest modul) 4. Cantitate : transporta doar TONE si este de 3 feluri : cantitate efectiva, cantitate tolerate (5, 10%) si cantitate la optiunea cuiva (comparator sau furnizor) 5. Livrare conditii: - incoterms („Incoterms” reprezintă prescurtarea pentru International Commercial Terms. Publicată pentru prima dată în 1936, reprezintă de fapt un set de 11 reguli, care definesc responsabilitatea în cadrul tranzacțiilor internaționale.) - Locatie ( aici se doreste onlista principala automatizata si posibilitatea de